# 前置

In [1]:
import pandas as pd
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows
import numpy as np

In [2]:
# 读取文件
file_path = r'D:/Ori/人群.xlsx'
sheet_name = 'Sheet1'
df = pd.read_excel(file_path, sheet_name=sheet_name)

# 整体数据汇总

In [3]:
# 渠道成本计算
def channel_cost(row):
    if row['渠道'] == '横幅':
        return 0
    elif row['渠道'] == '短信':
        return row['触达用户数'] * 0.027
    elif row['渠道'] == 'push':
        return row['触达用户数'] * 0.0008
    elif row['渠道'] == '猜喜':
        return row['点击用户数'] * 2
    else:
        return 0  # 默认返回0，防止出现未知渠道的情况

df['渠道成本'] = df.apply(channel_cost, axis=1)

In [4]:
# 总花费
df['总花费'] = df['渠道成本'] + df['券补金额']

In [5]:
# 聚合
people = df.groupby(['城市','是否门店新','渠道']).agg({
    '触达用户数': 'sum',
    '点击用户数': 'sum',
    '领券用户数': 'sum',
    '交易用户数': 'sum',
    '核销用户数': 'sum',
    '交易订单量': 'sum',
    '核销订单量': 'sum',
    '交易新用户数': 'sum',
    '券补金额': 'sum',
    '渠道成本': 'sum',
    '总花费':'sum'
}).reset_index()

In [6]:
# 一些复合指标添加
people['触达点击率'] = people['点击用户数'] / people['触达用户数']
people['点击转化率'] = people['交易用户数'] / people['点击用户数']
people['支付核销率'] = people['核销用户数'] / people['交易用户数']
people['团单CAC'] = people['总花费'] / people['交易订单量']
people['核销CAC'] = people['总花费'] / people['核销订单量']

In [7]:
# 首先，确保所有 inf 值替换为 np.nan
people.replace([np.inf, -np.inf], np.nan, inplace=True)

# 然后，用 0 填充所有 NaN 值
people.fillna(0, inplace=True)

In [8]:
# 保留两位小数
decimal_columns = ['渠道成本', '总花费','团单CAC','核销CAC']
people[decimal_columns] = people[decimal_columns].round(2)

# 百分比
rate_columns = ['触达点击率', '点击转化率', '支付核销率']
people[rate_columns] = people[rate_columns].apply(lambda x: (x * 100).round(2).astype(str) + '%')

In [9]:
# 创建一个新的Excel工作簿
wb = Workbook()

ws1 = wb.active
ws1.title = '整体数据汇总'

# 写入数据到新的Sheet
for r in dataframe_to_rows(people, index=False, header=True):
    ws1.append(r)

In [10]:
people.head()

,城市,是否门店新,渠道,触达用户数,点击用户数,领券用户数,交易用户数,核销用户数,交易订单量,核销订单量,交易新用户数,券补金额,渠道成本,总花费,触达点击率,点击转化率,支付核销率,团单CAC,核销CAC
0,hangzhou,0,短信,20329,64,52051,428,321,439,321,0,4390,548.88,4938.88,0.31%,668.75%,75.0%,11.25,15.39
1,hangzhou,1,push,1147957,1429,1574359,175,139,181,139,56,3620,918.37,4538.37,0.12%,12.25%,79.43%,25.07,32.65
2,hangzhou,1,猜喜,6132,10,10,0,1,0,1,0,0,12264.00,12264.00,0.16%,0.0%,0.0%,0.00,12264.00
3,hangzhou,1,短信,19739,22,44811,70,43,71,43,0,1420,532.95,1952.95,0.11%,318.18%,61.43%,27.51,45.42
4,hangzhou,1,站内横幅,177165,3758,481823,88,59,89,59,18,1780,0.00,1780.00,2.12%,2.34%,67.05%,20.00,30.17


# 用增线上渠道获客成本

In [11]:
people_copy = people

In [12]:
# 聚合
people_copy = df.groupby(['城市','是否门店新']).agg({
    '触达用户数': 'sum',
    '点击用户数': 'sum',
    '领券用户数': 'sum',
    '交易用户数': 'sum',
    '核销用户数': 'sum',
    '交易订单量': 'sum',
    '核销订单量': 'sum',
    '交易新用户数': 'sum',
    '券补金额': 'sum',
    '渠道成本': 'sum',
    '总花费':'sum'
}).reset_index()

In [13]:
# 一堆复合运算
people_copy['日均触达用户数'] =  people_copy['触达用户数'] / 7
people_copy['日均点击用户数'] =  people_copy['点击用户数'] / 7
people_copy['日均领券用户数'] =  people_copy['领券用户数'] / 7
people_copy['日均交易用户数'] =  people_copy['交易用户数'] / 7
people_copy['日均核销用户数'] =  people_copy['核销用户数'] / 7
people_copy['日均订单量'] =  people_copy['交易订单量'] / 7
people_copy['日均核销订单量'] =  people_copy['核销订单量'] / 7
people_copy['触达点击率'] =  people_copy['点击用户数'] / people_copy['触达用户数']
people_copy['点击转化率'] =  people_copy['交易用户数'] / people_copy['点击用户数']
people_copy['触达转化率'] =  people_copy['交易用户数'] / people_copy['触达用户数']
people_copy['支付核销率'] =  people_copy['核销用户数'] / people_copy['交易用户数']
people_copy['团单CAC'] = people_copy['总花费'] / people_copy['交易订单量']

In [14]:
# 首先，确保所有 inf 值替换为 np.nan
people_copy.replace([np.inf, -np.inf], np.nan, inplace=True)

# 然后，用 0 填充所有 NaN 值
people_copy.fillna(0, inplace=True)

In [15]:
# 保留两位小数
change_columns = ['日均触达用户数','日均点击用户数','日均领券用户数','日均交易用户数','日均核销用户数','日均订单量','日均核销订单量','团单CAC']
people_copy[change_columns] = people_copy[change_columns].round(2)

# 百分比
change2_columns = ['触达点击率', '点击转化率','触达转化率', '支付核销率']
people_copy[change2_columns] = people_copy[change2_columns].apply(lambda x: (x * 100).round(2).astype(str) + '%')

In [16]:
need_columns = ['城市','是否门店新','日均触达用户数','日均点击用户数'
                ,'日均领券用户数','日均交易用户数','日均核销用户数'
                ,'日均订单量','日均核销订单量','触达点击率'
                ,'点击转化率','触达转化率','支付核销率','团单CAC']
people_end = people_copy[need_columns]

In [17]:
ws2 = wb.create_sheet(title='用增线上渠道获客成本')
# 写入数据到新的Sheet
for r in dataframe_to_rows(people_end, index=False, header=True):
    ws2.append(r)

In [18]:
people_end.head()

,城市,是否门店新,日均触达用户数,日均点击用户数,日均领券用户数,日均交易用户数,日均核销用户数,日均订单量,日均核销订单量,触达点击率,点击转化率,触达转化率,支付核销率,团单CAC
0,hangzhou,0,2904.14,9.14,7435.86,61.14,45.86,62.71,45.86,0.31%,668.75%,2.11%,75.0%,11.25
1,hangzhou,1,192999.00,745.57,300143.29,47.57,34.57,48.71,34.57,0.39%,6.38%,0.02%,72.67%,60.22
2,shanghai,1,201450.86,305.43,280430.14,4.43,0.00,4.43,0.00,0.15%,1.45%,0.0%,0.0%,1631.93


In [19]:
# 保存新的Excel文件
output_file_path = r'C:\Users\1\Desktop\周报填写.xlsx'
wb.save(output_file_path)